In [ ]:
import os
import sys
import xarray as xr
import numpy as np
from numpy import unravel_index
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.font_manager import FontProperties
from datetime import datetime
from pathlib import Path
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib.patches import Rectangle

import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
# # set fonts configuration - Arial
# matplotlib.rcParams['font.family'] = "Open Sans"
# matplotlib.rcParams['font.sans-serif'] = "Arial"

# # get normal font and store it in `fira_sans_regular`
# personal_path = '/home/wmtsai/anaconda3/fonts/arial/'
# font_path = personal_path + 'ARIAL.TTF'
# arial_reg = FontProperties(fname=font_path)

In [ ]:
stats_dir = Path('/scratch/wmtsai/temp_mcs/output_stats/cape_subsat_hist/')

In [ ]:
data= xr.open_dataset('/scratch/wmtsai/temp_mcs/output_stats/era5_0.25deg_BL_2Dhistogram.wbwl.landsea.2001-2010.nc')

samples_clim_ocean = data.sel(surface_type='ocean')
samples_clim_land = data.sel(surface_type='land')

In [ ]:
%%time
year_list = np.arange(2001,2021)

for n,year in enumerate(year_list):
    
    data = xr.open_dataset(stats_dir / 'hist2d_cape_subsat_dtype.{}.ocean.FNL.alltracks.nc'.format(year))
    if n == 0:
        samples_all_oce = data.samples
        prec_gpm_sum_oce = data.prec_gpm_sum
        prec_era5_sum_oce = data.prec_era5_sum
    else:
        samples_all_oce = samples_all_oce + data.samples
        prec_gpm_sum_oce = prec_gpm_sum_oce + data.prec_gpm_sum
        prec_era5_sum_oce = prec_era5_sum_oce + data.prec_era5_sum

#### 2D BL histogram based on MCS grids during MCS phases
- mcs: sampling mcs grids
- over ocean

In [ ]:
fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(6,4.5))

for p,(ax,phase,label_str) in enumerate(zip([ax1,ax2,ax3,ax4,ax5,ax6],samples_all_oce.phase.values,
                                           ['(a)','(b)','(c)','(d)','(e)','(f)'])):
    
    gpm_sum = prec_gpm_sum_oce.sel(duration_type=['SL','ML','LL','UL','UUL'], area_type='mcs', phase=phase).sum('duration_type')
    tmp = samples_all_oce.sel(duration_type=['SL','ML','LL','UL','UUL'], area_type='mcs', phase=phase).sum('duration_type')
    tmp = tmp.where(tmp > 50)
    # conditional mean of gpm precip (mm/hr)
    cp = ax.pcolormesh(tmp.bins_subsat, tmp.bins_cape, gpm_sum/tmp, cmap='terrain_r', vmin=0, vmax=10, zorder=2)
    if phase == 'End':
        cbaxes = fig.add_axes([0.985, 0.3, 0.01, 0.4])              
        cbar = plt.colorbar(cp, cax=cbaxes, shrink=0.7, extend='max')
        cbar.set_label('(mm/hr)')
    
    ax.text(-1.5,4.5,'{} {}'.format(label_str,phase),fontsize=9,color='k')
    
    # contour highlighting light rainfall over low BL 
    #ax.contour(tmp.bins_subsat, tmp.bins_cape, gpm_sum/tmp, levels=[2.5], colors=['limegreen'])
    
    # joint probability
    #cf = ax.contour(tmp.bins_subsat, tmp.bins_cape, tmp/tmp.sum(), levels=np.arange(0.005,0.05,0.01), colors=['m'],
    #          linewidths=0.5, zorder=2)
    #ax.clabel(cf, cf.levels, fontsize=5, zorder=3)
    
    # add buoyancy reference 
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)+2, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)+4, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)+6, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='k', ls='-',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='k', ls='-',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)
    
    #ax.set_xticklabels([])
    #ax.set_yticklabels([])

    ## configuration of tick, labels
    ax.grid(ls=':', lw=0.5, zorder=1)
    if p == 3 or p == 4 or p == 5:
        ax.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
    ax.set_xticks([-2,0,2,4,6,8,10,12])
    if p == 0 or p == 3:
        ax.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
    ax.set_yticks([-8,-6,-4,-2,0,2,4])
    ax.tick_params(labelsize=9)
    ax.tick_params('both', length=0, width=1, which='major')
    
    # set x-y domain 
    ax.set_ylim([-8,6])
    ax.set_xlim([-2,14])  
    
ax1.set_xticklabels([])
ax2.set_xticklabels([])
ax2.set_yticklabels([])
ax3.set_xticklabels([])
ax3.set_yticklabels([])
ax5.set_yticklabels([])
ax6.set_yticklabels([])

plt.suptitle('Conditional GPM-precip, ocean, MCS grids \n (# samples in each bin > 50)',fontsize=10, fontweight='bold',y=0.97);
plt.tight_layout(h_pad=0.4,w_pad=0.01)

fig.savefig('/scratch/wmtsai/temp_mcs/fig/BL2Dhist_condiGPMprecip_phase_ocean.pdf',
       bbox_inches='tight',transparent=False)

In [ ]:
fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(6,4))

bins_cape = prec_gpm_sum_oce.bins_cape
bins_subsat = prec_gpm_sum_oce.bins_subsat

dbl = 0.5
prec_gpm_sum = prec_gpm_sum_oce.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))
samples_all = samples_all_oce.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))

# total samples and rain rate in all phases
samples_allphases = samples_all_oce.sel(area_type='mcs',phase=['Initial','End','Grow','Mature','Decay']).sum(('duration_type','phase'))
total_rain = prec_gpm_sum.sel(area_type='mcs',phase=['Initial','End','Grow','Mature','Decay']).sum(('duration_type','phase'))

for p,(ax,phase,label_str) in enumerate(zip([ax1,ax3,ax4,ax5,ax6],['Initial','End','Grow','Mature','Decay'],
                                            ['(a)','(b)','(c)','(d)','(e)'])):
    
    tmp = samples_all.sel(area_type='mcs', phase=phase).sum('duration_type')
    tmp = tmp.where(samples_allphases > 1000) # only if total samples > 100 
    
    # conditional mean of gpm precip (mm/hr)
    cp = ax.pcolormesh(tmp.bins_subsat, tmp.bins_cape, tmp/samples_allphases, cmap='CMRmap_r', vmin=0, vmax=0.5, zorder=2)
    if phase == 'End':
        cbaxes = fig.add_axes([0.985, 0.3, 0.01, 0.4])              
        cbar = plt.colorbar(cp, cax=cbaxes, shrink=0.7, extend='max')
        cbar.set_label('Fraction of occurrences')
    
    # add contour 
    ax.contour(samples_allphases.bins_subsat, samples_allphases.bins_cape, np.log10(samples_allphases), 
               colors=['k'], levels=[3,4,5,6], linewidths=0.5)   
        
    # add buoyancy reference 
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)
    
    # add mode mark
    (idx,idy) = unravel_index(samples_allphases.argmax(), samples_allphases.shape)
    cape_mode = samples_allphases.bins_cape[idx]
    subsat_mode = samples_allphases.bins_subsat[idy] 
    ax.plot(subsat_mode, cape_mode, 's', ms=4, color='grey')
    
    ## configuration of tick, labels
    ax.text(-1.2,4.3,'{}'.format(phase),fontsize=9,color='k')
    ax.grid(ls=':', lw=0.5, zorder=1)
    ax.set_xticks([-2,0,2,4,6,8,10,12])
    ax.set_yticks([-8,-6,-4,-2,0,2,4])
    ax.tick_params(labelsize=9)
    ax.tick_params('both', length=0, width=1, which='major')
    
    # set x-y domain 
    ax.set_ylim([-8,6])
    ax.set_xlim([-2,14])  
    
ax2.tick_params('both', length=0, width=1, which='major')
    
ax1.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax4.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax4.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
ax5.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
ax6.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)

ax1.set_xticklabels([])
ax2.set_xticklabels([])
ax2.set_yticklabels([])
ax3.set_xticklabels([])
ax3.set_yticklabels([])
ax5.set_yticklabels([])
ax6.set_yticklabels([])

#====== schematic part for axis 2 =======
# add buoyancy reference 
ax2.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1)
ax2.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='grey', ls='--',lw=0.5)
ax2.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='grey', ls='--',lw=0.5)
ax2.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)
ax2.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-8, color='grey', ls='--',lw=0.5)

(idx,idy) = unravel_index(samples_allphases.argmax(), samples_allphases.shape)
cape_mode = samples_allphases.bins_cape[idx]
subsat_mode = samples_allphases.bins_subsat[idy] 
ax2.plot(subsat_mode, cape_mode, 's', ms=4, color='grey')

ax2.set_ylim([-8,6])
ax2.set_xlim([-2,14])

plt.suptitle('Fraction of occurrences from MCS phases, MCS grids, Ocean',fontsize=10, fontweight='bold',y=0.97);
plt.tight_layout(h_pad=0.4,w_pad=0.01)

plt.show()

fig.savefig('/scratch/wmtsai/temp_mcs/fig/BL2Dhist_fraction_occurrence_ocean.rotate.pdf',
     bbox_inches='tight',transparent=False)

In [ ]:
#### contribution of total rainfall 

fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(6,4))

bins_cape = prec_gpm_sum_oce.bins_cape
bins_subsat = prec_gpm_sum_oce.bins_subsat

dbl = 0.5
prec_gpm_sum = prec_gpm_sum_oce.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))
samples_all = samples_all_oce.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))

# total samples and rain rate in all phases
samples_allphases = samples_all_oce.sel(area_type='mcs',phase=['Initial','End','Grow','Mature','Decay']).sum(('duration_type','phase'))
total_rain = prec_gpm_sum.sel(area_type='mcs',phase=['Initial','End','Grow','Mature','Decay']).sum(('duration_type','phase'))

for p,(ax,phase,label_str) in enumerate(zip([ax1,ax3,ax4,ax5,ax6],['Initial','End','Grow','Mature','Decay'],
                                            ['(a)','(b)','(c)','(d)','(e)'])):
    
    gpm_sum = prec_gpm_sum.sel(area_type='mcs', phase=phase).sum('duration_type')
    tmp = samples_all.sel(area_type='mcs', phase=phase).sum('duration_type')
    gpm_sum = gpm_sum.where(samples_allphases > 1000) # only if total samples > 100 
    
    # conditional mean of gpm precip (mm/hr)
    cp = ax.pcolormesh(tmp.bins_subsat, tmp.bins_cape, gpm_sum/total_rain, cmap='CMRmap_r', vmin=0, vmax=0.5, zorder=2)
    if phase == 'End':
        cbaxes = fig.add_axes([0.985, 0.3, 0.01, 0.4])              
        cbar = plt.colorbar(cp, cax=cbaxes, shrink=0.7, extend='max')
        cbar.set_label('Fraction of rainfall')
    
    # add contour 
    ax.contour(samples_allphases.bins_subsat, samples_allphases.bins_cape, np.log10(samples_allphases), 
               colors=['k'], levels=[3,4,5,6], linewidths=0.5)   
        
    # add buoyancy reference 
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)
    
    ## configuration of tick, labels
    ax.text(-1.2,4.3,'{}'.format(phase),fontsize=9,color='k')
    ax.grid(ls=':', lw=0.5, zorder=1)
    ax.set_xticks([-2,0,2,4,6,8,10,12])
    ax.set_yticks([-8,-6,-4,-2,0,2,4])
    ax.tick_params(labelsize=9)
    ax.tick_params('both', length=0, width=1, which='major')
    
    # set x-y domain 
    ax.set_ylim([-8,6])
    ax.set_xlim([-2,14])  
    
ax2.tick_params('both', length=0, width=1, which='major')
    
ax1.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax4.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax4.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
ax5.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
ax6.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)

ax1.set_xticklabels([])
ax2.set_xticklabels([])
ax2.set_yticklabels([])
ax3.set_xticklabels([])
ax3.set_yticklabels([])
ax5.set_yticklabels([])
ax6.set_yticklabels([])

#====== schematic part for axis 2 =======
# add buoyancy reference 
ax2.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1)
ax2.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='grey', ls='--',lw=0.5)
ax2.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='grey', ls='--',lw=0.5)
ax2.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)
ax2.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-8, color='grey', ls='--',lw=0.5)

(idx,idy) = unravel_index(samples_allphases.argmax(), samples_allphases.shape)
cape_mode = samples_allphases.bins_cape[idx]
subsat_mode = samples_allphases.bins_subsat[idy] 
ax2.plot(subsat_mode, cape_mode, 's', ms=4, color='grey')

ax2.set_ylim([-8,6])
ax2.set_xlim([-2,14])

plt.suptitle('Rainfall contribution from MCS phases, MCS grids, Ocean',fontsize=10, fontweight='bold',y=0.97);
plt.tight_layout(h_pad=0.4,w_pad=0.01)

plt.show()

fig.savefig('/scratch/wmtsai/temp_mcs/fig/BL2Dhist_GPMprecipContrib_phase_ocean.rotate.pdf',
      bbox_inches='tight',transparent=False)

#### over land

In [ ]:
%%time
year_list = np.arange(2001,2021)

for n,year in enumerate(year_list):
    
    data = xr.open_dataset(stats_dir / 'hist2d_cape_subsat_dtype.{}.land.FNL.alltracks.nc'.format(year))
    if n == 0:
        samples_all_land = data.samples
        prec_gpm_sum_land = data.prec_gpm_sum
        prec_era5_sum_land = data.prec_era5_sum
    else:
        samples_all_land = samples_all_land + data.samples
        prec_gpm_sum_land = prec_gpm_sum_land + data.prec_gpm_sum
        prec_era5_sum_land = prec_era5_sum_land + data.prec_era5_sum

In [ ]:
fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(6,4.5))

for p,(ax,phase,label_str) in enumerate(zip([ax1,ax2,ax3,ax4,ax5,ax6],samples_all_oce.phase.values,
                                           ['(a)','(b)','(c)','(d)','(e)','(f)'])):
    
    gpm_sum = prec_gpm_sum_land.sel(duration_type=['SL','ML','LL','UL','UUL'], area_type='mcs', phase=phase).sum('duration_type')
    tmp = samples_all_land.sel(duration_type=['SL','ML','LL','UL','UUL'], area_type='mcs', phase=phase).sum('duration_type')
    tmp = tmp.where(tmp > 50)
    # conditional mean of gpm precip (mm/hr)
    cp = ax.pcolormesh(tmp.bins_subsat, tmp.bins_cape, gpm_sum/tmp, cmap='terrain_r', vmin=0, vmax=5, zorder=2)
    if phase == 'End':
        cbaxes = fig.add_axes([0.985, 0.3, 0.01, 0.4])              
        cbar = plt.colorbar(cp, cax=cbaxes, shrink=0.7, extend='max')
        cbar.set_label('(mm/hr)')
    
    ax.text(-1.5,4.5,'{} {}'.format(label_str,phase),fontsize=9,color='k')
    
    # contour highlighting light rainfall over low BL 
    ax.contour(tmp.bins_subsat, tmp.bins_cape, gpm_sum/tmp, levels=[2.5], colors=['limegreen'])
    
    # joint probability
    #cf = ax.contour(tmp.bins_subsat, tmp.bins_cape, tmp/tmp.sum(), levels=np.arange(0.005,0.05,0.01), colors=['m'],
    #          linewidths=0.5, zorder=2)
    #ax.clabel(cf, cf.levels, fontsize=5, zorder=3)
    
    # add buoyancy reference 
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)+2, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)+4, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)+6, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='k', ls='-',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='k', ls='-',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)
    
    #ax.set_xticklabels([])
    #ax.set_yticklabels([])

    ## configuration of tick, labels
    ax.grid(ls=':', lw=0.5, zorder=1)
    if p == 3 or p == 4 or p == 5:
        ax.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
    ax.set_xticks([-2,0,2,4,6,8,10,12])
    if p == 0 or p == 3:
        ax.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
    ax.set_yticks([-8,-6,-4,-2,0,2,4])
    ax.tick_params(labelsize=9)
    ax.tick_params('both', length=0, width=1, which='major')
    
    # set x-y domain 
    ax.set_ylim([-8,6])
    ax.set_xlim([-2,14])  
    
ax1.set_xticklabels([])
ax2.set_xticklabels([])
ax2.set_yticklabels([])
ax3.set_xticklabels([])
ax3.set_yticklabels([])
ax5.set_yticklabels([])
ax6.set_yticklabels([])

plt.suptitle('Conditional GPM-precip, land, MCS grids \n (# samples in each bin > 50)',fontsize=10, fontweight='bold',y=0.97);
plt.tight_layout(h_pad=0.4,w_pad=0.01)

fig.savefig('/scratch/wmtsai/temp_mcs/fig/BL2Dhist_condiGPMprecip_phase_land.pdf',
       bbox_inches='tight',transparent=False)

#### Join ocean and land MCSs - lastest 

In [ ]:
%%time
year_list = np.arange(2001,2021)

for n,year in enumerate(year_list):
    
    data = xr.open_dataset(stats_dir / 'hist2d_cape_subsat_dtype.{}.land.FNL.alltracks.nc'.format(year))
    if n == 0:
        samples_all_land = data.samples
        prec_gpm_sum_land = data.prec_gpm_sum
        prec_era5_sum_land = data.prec_era5_sum
    else:
        samples_all_land = samples_all_land + data.samples
        prec_gpm_sum_land = prec_gpm_sum_land + data.prec_gpm_sum
        prec_era5_sum_land = prec_era5_sum_land + data.prec_era5_sum

for n,year in enumerate(year_list):
    
    data = xr.open_dataset(stats_dir / 'hist2d_cape_subsat_dtype.{}.ocean.FNL.alltracks.nc'.format(year))
    if n == 0:
        samples_all_oce = data.samples
        prec_gpm_sum_oce = data.prec_gpm_sum
        prec_era5_sum_oce = data.prec_era5_sum
    else:
        samples_all_oce = samples_all_oce + data.samples
        prec_gpm_sum_oce = prec_gpm_sum_oce + data.prec_gpm_sum
        prec_era5_sum_oce = prec_era5_sum_oce + data.prec_era5_sum
        
samples_all_comb = samples_all_land + samples_all_oce
prec_gpm_sum_comb = prec_gpm_sum_land + prec_gpm_sum_oce

In [ ]:
fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(6,4))

bins_cape = prec_gpm_sum_comb.bins_cape
bins_subsat = prec_gpm_sum_comb.bins_subsat

dbl = 0.25
prec_gpm_sum = prec_gpm_sum_comb.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))
samples_all = samples_all_comb.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))

# total samples and rain rate in all phases
samples_allphases = samples_all.sel(area_type='mcs',phase=['Initial','End','Grow','Mature','Decay']).sum(('duration_type','phase'))
total_rain = prec_gpm_sum.sel(area_type='mcs',phase=['Initial','End','Grow','Mature','Decay']).sum(('duration_type','phase'))

for p,(ax,phase,label_str) in enumerate(zip([ax2,ax3,ax4,ax5,ax6],['Initial','Grow','Mature','Decay','End'],
                                            ['(a)','(b)','(c)','(d)','(e)'])):
    
    gpm_sum = prec_gpm_sum.sel(area_type='mcs', phase=phase).sum('duration_type')
    tmp = samples_all.sel(area_type='mcs', phase=phase).sum('duration_type')
    tmp = tmp.where(samples_allphases > 1000) # only if total samples > 100 
    
    # conditional mean of gpm precip (mm/hr)
    #cp = ax.pcolormesh(tmp.bins_subsat, tmp.bins_cape, tmp/samples_allphases, cmap='CMRmap_r', vmin=0, vmax=0.5, zorder=2)
    cp = ax.contourf(tmp.bins_subsat, tmp.bins_cape, tmp/samples_allphases, cmap='CMRmap_r', levels=np.linspace(0,0.5,21), zorder=2, extend='both')    
    cs = ax.contour(tmp.bins_subsat, tmp.bins_cape, tmp/samples_allphases, colors=['k'],linewidths=0.5,levels=[0.2,0.25,0.3], zorder=2)    

    if phase == 'End':
        cbaxes = fig.add_axes([0.985, 0.3, 0.01, 0.4])              
        cbar = plt.colorbar(cp, cax=cbaxes, shrink=0.7)
        cbar.set_label('Fraction of occurrences')
        cbar.set_ticks([0,0.1,0.2,0.3,0.4,0.5])
    ax.clabel(cs, inline=False, fontsize=6)
    
    # # add contour 
    ax.contour(samples_allphases.bins_subsat, samples_allphases.bins_cape, np.log10(samples_allphases), 
               colors=['gray'], levels=[3,4,5,6], linewidths=0.5)   
        
    # add buoyancy reference 
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)
    
    # add mode
    (idx,idy) = unravel_index(samples_allphases.argmax(), samples_allphases.shape)
    cape_mode = samples_allphases.bins_cape[idx]
    subsat_mode = samples_allphases.bins_subsat[idy] 
    ax.plot(subsat_mode, cape_mode, 's', ms=4, color='grey')
    
    ## configuration of tick, labels
    ax.text(-1.2,4.3,'{}'.format(phase),fontsize=9,color='k')
    ax.grid(ls=':', lw=0.5, zorder=1)
    ax.set_xticks([-2,0,2,4,6,8,10,12])
    ax.set_yticks([-8,-6,-4,-2,0,2,4])
    ax.tick_params(labelsize=9)
    ax.tick_params('both', length=0, width=1, which='major')
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(1.5)
    
    # set x-y domain 
    ax.set_ylim([-8,6])
    ax.set_xlim([-2,14])  
    
ax1.tick_params('both', length=0, width=1, which='major')
    
ax1.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax4.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax4.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
ax5.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
ax6.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)

ax1.set_xticklabels([])
ax2.set_xticklabels([])
ax2.set_yticklabels([])
ax3.set_xticklabels([])
ax3.set_yticklabels([])
ax5.set_yticklabels([])
ax6.set_yticklabels([])

#====== schematic part for axis 3 =======
# add buoyancy reference 
ax1.plot(np.linspace(-2,16,13),np.linspace(-2,16,13), color='k', lw=1)
ax1.plot(np.linspace(-2,16,13),np.linspace(-2,16,13)-2, color='grey', ls='--',lw=0.5)
ax1.plot(np.linspace(-2,16,13),np.linspace(-2,16,13)-4, color='grey', ls='--',lw=0.5)
ax1.plot(np.linspace(-2,16,13),np.linspace(-2,16,13)-6, color='grey', ls='--',lw=0.5)

# (idx,idy) = unravel_index(samples_allphases.argmax(), samples_allphases.shape)
# cape_mode = samples_allphases.bins_cape[idx]
# subsat_mode = samples_allphases.bins_subsat[idy] 
# ax2.plot(subsat_mode, cape_mode, 's', ms=4, color='grey')

ax1.set_ylim([-8,6])
ax1.set_xlim([-2,14])
ax1.set_xticks([-2,0,2,4,6,8,10,12])
ax1.set_yticks([-8,-6,-4,-2,0,2,4])

plt.suptitle('Fraction of occurrences, MCS grids, All',fontsize=10, fontweight='bold',y=0.97);
plt.tight_layout(h_pad=0.4,w_pad=0.01)

plt.show()

fig.savefig('/scratch/wmtsai/temp_mcs/fig/BL2Dhist_fraction_occurrence_allsurface.seqential.pdf',
     bbox_inches='tight',transparent=False)

In [ ]:
fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(6,4))

bins_cape = prec_gpm_sum_comb.bins_cape
bins_subsat = prec_gpm_sum_comb.bins_subsat

dbl = 0.25
prec_gpm_sum = prec_gpm_sum_comb.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))
samples_all = samples_all_comb.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))

# total samples and rain rate in all phases
samples_allphases = samples_all.sel(area_type='mcs',phase=['Initial','End','Grow','Mature','Decay']).sum(('duration_type','phase'))
total_rain = prec_gpm_sum.sel(area_type='mcs',phase=['Initial','End','Grow','Mature','Decay']).sum(('duration_type','phase'))

for p,(ax,phase,label_str) in enumerate(zip([ax1,ax2,ax4,ax5,ax6],['Initial','Grow','Mature','Decay','End'],
                                            ['(a)','(b)','(c)','(d)','(e)'])):
    
    gpm_sum = prec_gpm_sum.sel(area_type='mcs', phase=phase).sum('duration_type')
    gpm_sum = gpm_sum.where(samples_allphases > 1000) # only if total samples > 1000
    tmp = samples_all.sel(area_type='mcs', phase=phase).sum('duration_type')
    tmp = tmp.where(samples_allphases > 1000) # only if total samples > 1000
    
    # conditional mean of gpm precip (mm/hr)
    #cp = ax.pcolormesh(tmp.bins_subsat, tmp.bins_cape, tmp/samples_allphases, cmap='CMRmap_r', vmin=0, vmax=0.5, zorder=2)
    cp = ax.contourf(tmp.bins_subsat, tmp.bins_cape, gpm_sum/total_rain, cmap='CMRmap_r', levels=np.linspace(0,0.5,21), zorder=2, extend='both')    
    cs = ax.contour(tmp.bins_subsat, tmp.bins_cape, gpm_sum/total_rain, colors=['k'],linewidths=0.5,levels=[0.2,0.25,0.3,0.35,0.4,0.45,0.5], zorder=2)    

    if phase == 'End':
        cbaxes = fig.add_axes([0.985, 0.3, 0.01, 0.4])              
        cbar = plt.colorbar(cp, cax=cbaxes, shrink=0.7)
        cbar.set_label('Rainfall contribution')
        cbar.set_ticks([0,0.1,0.2,0.3,0.4,0.5])
    ax.clabel(cs, inline=False, fontsize=6)
    
    # # add contour 
    tmp2 = samples_allphases.copy()
    tmp2 = tmp2.where(tmp2 >= 1200)
    ax.contour(samples_allphases.bins_subsat, samples_allphases.bins_cape, np.log10(samples_allphases), 
               colors=['gray'], levels=[3,4,5,6], linewidths=0.5)   
        
    # add buoyancy reference 
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)
    
    # add mode
    (idx,idy) = unravel_index(samples_allphases.argmax(), samples_allphases.shape)
    cape_mode = samples_allphases.bins_cape[idx]
    subsat_mode = samples_allphases.bins_subsat[idy] 
    ax.plot(subsat_mode, cape_mode, 's', ms=4, color='grey')
    
    ## configuration of tick, labels
    ax.text(-1.2,4.3,'{}'.format(phase),fontsize=9,color='k')
    ax.grid(ls=':', lw=0.5, zorder=1)
    ax.set_xticks([-2,0,2,4,6,8,10,12])
    ax.set_yticks([-8,-6,-4,-2,0,2,4])
    ax.tick_params(labelsize=9)
    ax.tick_params('both', length=0, width=1, which='major')
    
    # set x-y domain 
    ax.set_ylim([-8,6])
    ax.set_xlim([-2,14])  
        
ax1.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax4.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax4.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
ax5.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
ax6.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)

ax1.set_xticklabels([])
ax2.set_xticklabels([])
ax2.set_yticklabels([])
ax3.set_xticklabels([])
ax3.set_yticklabels([])
ax5.set_yticklabels([])
ax6.set_yticklabels([])

#====== schematic part for axis 2 =======
# add buoyancy reference 
ax3.tick_params('both', length=0, width=1, which='major')
ax3.plot(np.linspace(-2,16,13),np.linspace(-2,16,13), color='k', lw=1)
ax3.plot(np.linspace(-2,16,13),np.linspace(-2,16,13)-2, color='grey', ls='--',lw=0.5)
ax3.plot(np.linspace(-2,16,13),np.linspace(-2,16,13)-4, color='grey', ls='--',lw=0.5)
ax3.plot(np.linspace(-2,16,13),np.linspace(-2,16,13)-6, color='grey', ls='--',lw=0.5)

# (idx,idy) = unravel_index(samples_allphases.argmax(), samples_allphases.shape)
# cape_mode = samples_allphases.bins_cape[idx]
# subsat_mode = samples_allphases.bins_subsat[idy] 
# ax2.plot(subsat_mode, cape_mode, 's', ms=4, color='grey')

ax3.set_ylim([-8,6])
ax3.set_xlim([-2,14])

plt.suptitle('Rainfall contribution, MCS grids, All',fontsize=10, fontweight='bold',y=0.97);
plt.tight_layout(h_pad=0.4,w_pad=0.01)

plt.show()

fig.savefig('/scratch/wmtsai/temp_mcs/fig/BL2Dhist_GPMprecipContrib_allsurface.sequential.pdf',
   bbox_inches='tight',transparent=False)

In [ ]:
colors2 = plt.cm.YlGnBu(np.linspace(0.2, 1, 50))
# combine them and build a new colormap
mixedmap = LinearSegmentedColormap.from_list('mycolor',colors2)
mixedmap

In [ ]:
#### contribution of total rainfall 

fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(6,4))

bins_cape = prec_gpm_sum_comb.bins_cape
bins_subsat = prec_gpm_sum_comb.bins_subsat

dbl = 0.25
prec_gpm_sum = prec_gpm_sum_comb.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))
samples_all = samples_all_comb.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))

# total samples and rain rate in all phases
samples_allphases = samples_all_comb.sel(area_type='mcs',phase=['Initial','End','Grow','Mature','Decay']).sum(('duration_type','phase'))
total_rain = prec_gpm_sum.sel(area_type='mcs',phase=['Initial','End','Grow','Mature','Decay']).sum(('duration_type','phase'))

for p,(ax,phase,label_str) in enumerate(zip([ax2,ax3,ax4,ax5,ax6],['Initial','Grow','Mature','Decay','End'],
                                            ['(a)','(b)','(c)','(d)','(e)'])):
    
    gpm_sum = prec_gpm_sum_comb.sel(area_type='mcs', phase=phase).sum('duration_type')
    tmp = samples_all_comb.sel(area_type='mcs', phase=phase).sum('duration_type')
    tmp = tmp.where(tmp > 200)
    
    # conditional mean of gpm precip (mm/hr)
    cp = ax.contourf(tmp.bins_subsat, tmp.bins_cape, gpm_sum/tmp, cmap=mixedmap, levels=np.linspace(0,8,17), zorder=2, extend='max')
    ax.contour(tmp.bins_subsat, tmp.bins_cape, gpm_sum/tmp, colors='k', levels=[2,4,6,8,10], zorder=2, linewidths=0.5)

    if phase == 'End':
        cbaxes = fig.add_axes([0.985, 0.3, 0.01, 0.4])              
        cbar = plt.colorbar(cp, cax=cbaxes, shrink=0.7)
        cbar.set_label('(mm/hr)') 
        cbar.set_ticks([0,2,4,6,8])
        
    # add buoyancy reference 
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)
    
    ## configuration of tick, labels
    ax.text(-1.2,4.3,'{}'.format(phase),fontsize=9,color='k')
    ax.grid(ls=':', lw=0.5, zorder=1)
    ax.set_xticks([-2,0,2,4,6,8,10,12])
    ax.set_yticks([-8,-6,-4,-2,0,2,4])
    ax.tick_params(labelsize=9)
    ax.tick_params('both', length=0, width=1, which='major')
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(1.5)
    
    # set x-y domain 
    ax.set_ylim([-8,6])
    ax.set_xlim([-2,14])  
        
ax1.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax4.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax4.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
ax5.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
ax6.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)

ax1.set_xticklabels([])
ax2.set_xticklabels([])
ax2.set_yticklabels([])
ax3.set_xticklabels([])
ax3.set_yticklabels([])
ax5.set_yticklabels([])
ax6.set_yticklabels([])

#====== schematic part for axis 3 =======

# load data
ds_bins = xr.open_dataset('/scratch/wmtsai/temp_mcs/output_stats/BLhist2d_precipitation_uncondition.nc')
bins_cape = ds_bins.bins_cape
bins_subsat = ds_bins.bins_subsat

dbl = 0.25
prec_gpm_sum = ds_bins.prec_gpm_sum.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))
samples_all = ds_bins.samples.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))
prec_gpm_sum = prec_gpm_sum.where(samples_all > 200)

# conditional mean of gpm precip (mm/hr)
cp = ax1.contourf(samples_all.bins_subsat, samples_all.bins_cape, prec_gpm_sum/samples_all, cmap=mixedmap,levels=np.linspace(0,8,17), zorder=2, extend='max')
ax1.contour(samples_all.bins_subsat, samples_all.bins_cape, prec_gpm_sum/samples_all, colors='k', levels=[2,4,6,8,10], zorder=2, linewidths=0.5)

# add buoyancy reference 
ax1.text(-1.2,4.3,'{}'.format('Ref. (Tropical grids)'),fontsize=9,color='k')

ax1.tick_params('both', length=0, width=1, which='major')
ax1.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1)
ax1.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='grey', ls='--',lw=0.5)
ax1.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='grey', ls='--',lw=0.5)
ax1.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)

ax1.set_ylim([-8,6])
ax1.set_xlim([-2,14])
ax1.set_xticks([-2,0,2,4,6,8,10,12])
ax1.set_yticks([-8,-6,-4,-2,0,2,4])

plt.suptitle('Conditional precipitation, MCS grids, All \n N. of samples min = 200.', x=0.54, y=0.94, fontsize=9.5, fontweight='bold')
plt.tight_layout(h_pad=0.4,w_pad=0.01)

plt.show()

fig.savefig('/scratch/wmtsai/temp_mcs/fig/BL2Dhist_CondPrec_allsurface.sequential.reference.pdf',
   bbox_inches='tight',transparent=False)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(3.5,3.5))

bins_cape = prec_gpm_sum_comb.bins_cape
bins_subsat = prec_gpm_sum_comb.bins_subsat
ccolors = ['k','red','blue']

dbl = 0.25
prec_gpm_sum = prec_gpm_sum_comb.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))
samples_all = samples_all_comb.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))

for p,(phase,color) in enumerate(zip(['Initial','Mature','End'],ccolors)):
    
    tmp = samples_all.sel(area_type='mcs', phase=phase).sum('duration_type')
    #tmp = tmp.where(tmp > 100) # only if total samples > 100 
    
    #ax.contour(tmp.bins_subsat, tmp.bins_cape, np.log10(tmp), colors=color, levels=[2,3,4,5], 
    #          linewidths=0.5)   
    ax.contour(tmp.bins_subsat, tmp.bins_cape, np.log10(tmp/tmp.sum()), colors=color, levels=[-5,-4,-3,-2,-1], 
              linewidths=0.5)   
        
    # add buoyancy reference 
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)
    
    # add mode of each mode
    (idx,idy) = unravel_index(tmp.argmax(), tmp.shape)
    cape_mode = tmp.bins_cape[idx]
    subsat_mode = tmp.bins_subsat[idy] 
    ax.plot(subsat_mode, cape_mode, 's', ms=3, color=color, alpha=0.5)
    
ax.grid(ls=':', lw=0.5, zorder=1)
ax.set_xticks([-2,0,2,4,6,8,10,12])
ax.set_yticks([-8,-6,-4,-2,0,2,4])
ax.tick_params(labelsize=9)
ax.tick_params('both', length=0, width=1, which='major')

# set x-y domain 
ax.set_ylim([-8,6])
ax.set_xlim([-2,14])  
ax.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)

plt.show()

#fig.savefig('/scratch/wmtsai/temp_mcs/fig/BL2Dhist_fraction_occurrence_allsurface.rotate.D12to24.pdf',
#   bbox_inches='tight',transparent=False)

#### non-MCS cases - explore diff. in conditional precip. 

In [ ]:
year_list = np.arange(2010,2012)

for n,year in enumerate(year_list):
    
    data = xr.open_dataset(stats_dir / 'hist2d_cape_subsat_dtype.{}.all.FNL.nonmcs.nc'.format(year))
    if n == 0:
        samples_all_nonmcs = data.samples
        prec_gpm_sum_nonmcs = data.prec_gpm_sum
        prec_era5_sum_nonmcs = data.prec_era5_sum
    else:
        samples_all_nonmcs = samples_all_land + data.samples
        prec_gpm_sum_nonmcs = prec_gpm_sum_land + data.prec_gpm_sum
        prec_era5_sum_nonmcs = prec_era5_sum_land + data.prec_era5_sum

In [ ]:
fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(6,4))

bins_cape = prec_gpm_sum_nonmcs.bins_cape
bins_subsat = prec_gpm_sum_nonmcs.bins_subsat

dbl = 0.25
prec_gpm_sum = prec_gpm_sum_nonmcs.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))
samples_all = samples_all_nonmcs.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))

# total samples and rain rate in all phases
samples_allphases = samples_all_nonmcs.sel(area_type='mcs',phase=['Initial','End','Grow','Mature','Decay']).sum(('duration_type','phase'))
total_rain = prec_gpm_sum.sel(area_type='mcs',phase=['Initial','End','Grow','Mature','Decay']).sum(('duration_type','phase'))

for p,(ax,phase,label_str) in enumerate(zip([ax1,ax3,ax4,ax5,ax6],['Initial','End','Grow','Mature','Decay'],
                                            ['(a)','(b)','(c)','(d)','(e)'])):
    
    gpm_sum = prec_gpm_sum.sel(area_type='mcs', phase=phase).sum('duration_type')
    tmp = samples_all.sel(area_type='mcs', phase=phase).sum('duration_type')
    tmp = tmp.where(samples_allphases > 1000) # only if total samples > 100 
    
    # conditional mean of gpm precip (mm/hr)
    #cp = ax.pcolormesh(tmp.bins_subsat, tmp.bins_cape, tmp/samples_allphases, cmap='CMRmap_r', vmin=0, vmax=0.5, zorder=2)
    cp = ax.contourf(tmp.bins_subsat, tmp.bins_cape, tmp/samples_allphases, cmap='jet', levels=np.linspace(0,0.4,21), zorder=2, extend='both')    
    #ax.contour(tmp.bins_subsat, tmp.bins_cape, tmp/samples_allphases, colors=['k'],linewidths=0.5,levels=[0.2,0.3,0.4], zorder=2)    

    if phase == 'End':
        cbaxes = fig.add_axes([0.985, 0.3, 0.01, 0.4])              
        cbar = plt.colorbar(cp, cax=cbaxes, shrink=0.7)
        cbar.set_label('Fraction of occurrences')
    
    # # add contour 
    ax.contour(samples_allphases.bins_subsat, samples_allphases.bins_cape, np.log10(samples_allphases), 
               colors=['k'], levels=[3,4,5,6], linewidths=0.5)   
        
    # add buoyancy reference 
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)
    
    # add mode
    (idx,idy) = unravel_index(samples_allphases.argmax(), samples_allphases.shape)
    cape_mode = samples_allphases.bins_cape[idx]
    subsat_mode = samples_allphases.bins_subsat[idy] 
    ax.plot(subsat_mode, cape_mode, 's', ms=4, color='grey')
    
    ## configuration of tick, labels
    ax.text(-1.2,4.3,'{}'.format(phase),fontsize=9,color='k')
    ax.grid(ls=':', lw=0.5, zorder=1)
    ax.set_xticks([-2,0,2,4,6,8,10,12])
    ax.set_yticks([-8,-6,-4,-2,0,2,4])
    ax.tick_params(labelsize=9)
    ax.tick_params('both', length=0, width=1, which='major')
    
    # set x-y domain 
    ax.set_ylim([-8,6])
    ax.set_xlim([-2,14])  
    
ax2.tick_params('both', length=0, width=1, which='major')
    
ax1.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax4.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax4.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
ax5.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
ax6.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)

ax1.set_xticklabels([])
ax2.set_xticklabels([])
ax2.set_yticklabels([])
ax3.set_xticklabels([])
ax3.set_yticklabels([])
ax5.set_yticklabels([])
ax6.set_yticklabels([])

#====== schematic part for axis 2 =======
# add buoyancy reference 
ax2.plot(np.linspace(-2,16,13),np.linspace(-2,16,13), color='k', lw=1)
ax2.plot(np.linspace(-2,16,13),np.linspace(-2,16,13)-2, color='grey', ls='--',lw=0.5)
ax2.plot(np.linspace(-2,16,13),np.linspace(-2,16,13)-4, color='grey', ls='--',lw=0.5)
ax2.plot(np.linspace(-2,16,13),np.linspace(-2,16,13)-6, color='grey', ls='--',lw=0.5)

# (idx,idy) = unravel_index(samples_allphases.argmax(), samples_allphases.shape)
# cape_mode = samples_allphases.bins_cape[idx]
# subsat_mode = samples_allphases.bins_subsat[idy] 
# ax2.plot(subsat_mode, cape_mode, 's', ms=4, color='grey')

ax2.set_ylim([-8,6])
ax2.set_xlim([-2,14])

plt.suptitle('Fraction of occurrences, Small-scale grids, All',fontsize=10, fontweight='bold',y=0.97);
plt.tight_layout(h_pad=0.4,w_pad=0.01)

plt.show()

#fig.savefig('/scratch/wmtsai/temp_mcs/fig/BL2Dhist_fraction_occurrence_allsurface.rotate.pdf',
#    bbox_inches='tight',transparent=False)

In [ ]:
#### contribution of total rainfall 

fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(6,4))

bins_cape = prec_gpm_sum_nonmcs.bins_cape
bins_subsat = prec_gpm_sum_nonmcs.bins_subsat

dbl = 0.25
prec_gpm_sum = prec_gpm_sum_nonmcs.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))
samples_all = samples_all_nonmcs.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))

# total samples and rain rate in all phases
samples_allphases = samples_all_nonmcs.sel(area_type='mcs',phase=['Initial','End','Grow','Mature','Decay']).sum(('duration_type','phase'))
total_rain = prec_gpm_sum.sel(area_type='mcs',phase=['Initial','End','Grow','Mature','Decay']).sum(('duration_type','phase'))

for p,(ax,phase,label_str) in enumerate(zip([ax1,ax3,ax4,ax5,ax6],['Initial','End','Grow','Mature','Decay'],
                                            ['(a)','(b)','(c)','(d)','(e)'])):
    
    gpm_sum = prec_gpm_sum_nonmcs.sel(area_type='mcs', phase=phase).sum('duration_type')
    tmp = samples_all_nonmcs.sel(area_type='mcs', phase=phase).sum('duration_type')
    tmp = tmp.where(tmp > 100)
    
    # conditional mean of gpm precip (mm/hr)
    cp = ax.contourf(tmp.bins_subsat, tmp.bins_cape, gpm_sum/tmp, cmap='YlGnBu', levels=np.linspace(0,8,17), zorder=2, extend='max')
    ax.contour(tmp.bins_subsat, tmp.bins_cape, gpm_sum/tmp, colors='k', levels=[2], zorder=2, linewidths=1)
    ax.contour(tmp.bins_subsat, tmp.bins_cape, gpm_sum/tmp, colors='k', levels=[4,6,8], zorder=2, linewidths=0.5)

    if phase == 'End':
        cbaxes = fig.add_axes([0.985, 0.3, 0.01, 0.4])              
        cbar = plt.colorbar(cp, cax=cbaxes, shrink=0.7)
        cbar.set_label('(mm/hr)') 
        cbar.set_ticks([0,2,4,6,8])
        
    # add buoyancy reference 
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)
    
    ## configuration of tick, labels
    ax.text(-1.2,4.3,'{}'.format(phase),fontsize=9,color='k')
    ax.grid(ls=':', lw=0.5, zorder=1)
    ax.set_xticks([-2,0,2,4,6,8,10,12])
    ax.set_yticks([-8,-6,-4,-2,0,2,4])
    ax.tick_params(labelsize=9)
    ax.tick_params('both', length=0, width=1, which='major')
    
    # set x-y domain 
    ax.set_ylim([-8,6])
    ax.set_xlim([-2,14])  
    
ax2.tick_params('both', length=0, width=1, which='major')
    
ax1.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax4.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax4.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
ax5.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
ax6.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)

ax1.set_xticklabels([])
ax2.set_xticklabels([])
ax2.set_yticklabels([])
ax3.set_xticklabels([])
ax3.set_yticklabels([])
ax5.set_yticklabels([])
ax6.set_yticklabels([])

#====== schematic part for axis 2 =======
# add buoyancy reference 
ax2.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1)
ax2.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='grey', ls='--',lw=0.5)
ax2.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='grey', ls='--',lw=0.5)
ax2.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)

(idx,idy) = unravel_index(samples_allphases.argmax(), samples_allphases.shape)
cape_mode = samples_allphases.bins_cape[idx]
subsat_mode = samples_allphases.bins_subsat[idy] 
ax2.plot(subsat_mode, cape_mode, 's', ms=4, color='grey')

ax2.set_ylim([-8,6])
ax2.set_xlim([-2,14])

plt.suptitle('Conditional precipitation, Small-scale, All', x=0.54, y=0.94, fontsize=9.5, fontweight='bold')
plt.tight_layout(h_pad=0.4,w_pad=0.01)

plt.show()

# fig.savefig('/scratch/wmtsai/temp_mcs/fig/BL2Dhist_GPMprecipContrib_allsurface.rotate.pdf',
#      bbox_inches='tight',transparent=False)

#### MCS mean value

In [ ]:
fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(6,4))

bins_cape = prec_gpm_sum_oce.bins_cape
bins_subsat = prec_gpm_sum_oce.bins_subsat

dbl = 0.25
prec_gpm_sum = prec_gpm_sum_oce.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))
samples_all = samples_all_oce.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))

# total samples and rain rate in all phases
samples_allphases = samples_all_oce.sel(area_type='mcs_mean',phase=['Initial','End','Grow','Mature','Decay']).sum(('duration_type','phase'))
total_rain = prec_gpm_sum.sel(area_type='mcs_mean',phase=['Initial','End','Grow','Mature','Decay']).sum(('duration_type','phase'))

for p,(ax,phase,label_str) in enumerate(zip([ax1,ax3,ax4,ax5,ax6],['Initial','End','Grow','Mature','Decay'],
                                            ['(a)','(b)','(c)','(d)','(e)'])):
    
    gpm_sum = prec_gpm_sum.sel(area_type='mcs_mean', phase=phase).sum('duration_type')
    tmp = samples_all.sel(area_type='mcs_mean', phase=phase).sum('duration_type')
    tmp = tmp.where(samples_allphases > 50) # only if total samples > 100 
    
    # conditional mean of gpm precip (mm/hr)
    #cp = ax.pcolormesh(tmp.bins_subsat, tmp.bins_cape, tmp/samples_allphases, cmap='CMRmap_r', vmin=0, vmax=0.5, zorder=2)
    cp = ax.contourf(tmp.bins_subsat, tmp.bins_cape, tmp/samples_allphases, cmap='jet', levels=np.linspace(0,0.5,21), zorder=2, extend='both')    
    #ax.contour(tmp.bins_subsat, tmp.bins_cape, tmp/samples_allphases, colors=['k'],linewidths=0.5,levels=[0.2,0.3,0.4], zorder=2)    

    if phase == 'End':
        cbaxes = fig.add_axes([0.985, 0.3, 0.01, 0.4])              
        cbar = plt.colorbar(cp, cax=cbaxes, shrink=0.7)
        cbar.set_label('Fraction of occurrences')
    
    # # add contour 
    ax.contour(samples_allphases.bins_subsat, samples_allphases.bins_cape, np.log10(samples_allphases), 
               colors=['k'], levels=[2,2.5,3], linewidths=0.5)   
        
    # add buoyancy reference 
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)
    
    # add mode
    (idx,idy) = unravel_index(samples_allphases.argmax(), samples_allphases.shape)
    cape_mode = samples_allphases.bins_cape[idx]
    subsat_mode = samples_allphases.bins_subsat[idy] 
    ax.plot(subsat_mode, cape_mode, 's', ms=4, color='grey')
    
    ## configuration of tick, labels
    ax.text(0.3,2.8,'{}'.format(phase),fontsize=9,color='k')
    ax.grid(ls=':', lw=0.5, zorder=1)
    ax.set_xticks([-2,0,2,4,6,8,10,12])
    ax.set_yticks([-8,-6,-4,-2,0,2,4])
    ax.tick_params(labelsize=9)
    ax.tick_params('both', length=0, width=1, which='major')
    
    # set x-y domain 
    ax.set_ylim([-3,4])
    ax.set_xlim([0,8])
    
ax2.tick_params('both', length=0, width=1, which='major')
    
ax1.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax4.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax4.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
ax5.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
ax6.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)

ax1.set_xticklabels([])
ax2.set_xticklabels([])
ax2.set_yticklabels([])
ax3.set_xticklabels([])
ax3.set_yticklabels([])
ax5.set_yticklabels([])
ax6.set_yticklabels([])

#====== schematic part for axis 2 =======
# add buoyancy reference 
ax2.plot(np.linspace(-2,16,13),np.linspace(-2,16,13), color='k', lw=1)
ax2.plot(np.linspace(-2,16,13),np.linspace(-2,16,13)-2, color='grey', ls='--',lw=0.5)
ax2.plot(np.linspace(-2,16,13),np.linspace(-2,16,13)-4, color='grey', ls='--',lw=0.5)
ax2.plot(np.linspace(-2,16,13),np.linspace(-2,16,13)-6, color='grey', ls='--',lw=0.5)

(idx,idy) = unravel_index(samples_allphases.argmax(), samples_allphases.shape)
cape_mode = samples_allphases.bins_cape[idx]
subsat_mode = samples_allphases.bins_subsat[idy] 
ax2.plot(subsat_mode, cape_mode, 's', ms=4, color='grey')

ax2.set_ylim([-3,4])
ax2.set_xlim([0,8])

plt.suptitle('Fraction of occurrences, MCS mean, All',fontsize=10, fontweight='bold',y=0.97);
plt.tight_layout(h_pad=0.4,w_pad=0.01)
plt.show()

#fig.savefig('/scratch/wmtsai/temp_mcs/fig/BL2Dhist_fraction_occurrence_allsurface.rotate.pdf',
#     bbox_inches='tight',transparent=False)

#### seprarating by duration type

In [ ]:
fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(6,4))

bins_cape = prec_gpm_sum_comb.bins_cape
bins_subsat = prec_gpm_sum_comb.bins_subsat

dbl = 0.25
prec_gpm_sum = prec_gpm_sum_comb.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))
samples_all = samples_all_comb.interp(bins_cape=np.arange(bins_cape[0],bins_cape[-1]+dbl,dbl),
                                           bins_subsat=np.arange(bins_subsat[0],bins_subsat[-1]+dbl,dbl))

# total samples and rain rate in all phases
samples_allphases = samples_all_comb.sel(area_type='mcs',phase=['Initial','End','Grow','Mature','Decay'],duration_type=['LL','UL']).sum(('duration_type','phase'))
total_rain = prec_gpm_sum.sel(area_type='mcs',phase=['Initial','End','Grow','Mature','Decay'],duration_type=['LL','UL']).sum(('duration_type','phase'))

for p,(ax,phase,label_str) in enumerate(zip([ax1,ax3,ax4,ax5,ax6],['Initial','End','Grow','Mature','Decay'],
                                            ['(a)','(b)','(c)','(d)','(e)'])):
    
    gpm_sum = prec_gpm_sum.sel(area_type='mcs', phase=phase, duration_type=['LL','UL']).sum('duration_type')
    tmp = samples_all.sel(area_type='mcs', phase=phase, duration_type=['LL','UL']).sum('duration_type')
    tmp = tmp.where(samples_allphases > 1000) # only if total samples > 100 
    
    # conditional mean of gpm precip (mm/hr)
    #cp = ax.pcolormesh(tmp.bins_subsat, tmp.bins_cape, tmp/samples_allphases, cmap='CMRmap_r', vmin=0, vmax=0.5, zorder=2)
    cp = ax.contourf(tmp.bins_subsat, tmp.bins_cape, tmp/samples_allphases, cmap='jet', levels=np.linspace(0,0.4,21), zorder=2, extend='both')    
    #ax.contour(tmp.bins_subsat, tmp.bins_cape, tmp/samples_allphases, colors=['k'],linewidths=0.5,levels=[0.2,0.3,0.4], zorder=2)    

    if phase == 'End':
        cbaxes = fig.add_axes([0.985, 0.3, 0.01, 0.4])              
        cbar = plt.colorbar(cp, cax=cbaxes, shrink=0.7)
        cbar.set_label('Fraction of occurrences')
    
    # # add contour 
    ax.contour(samples_allphases.bins_subsat, samples_allphases.bins_cape, np.log10(samples_allphases), 
               colors=['k'], levels=[3,4,5,6], linewidths=0.5)   
        
    # add buoyancy reference 
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)
    
    # add mode
    (idx,idy) = unravel_index(samples_allphases.argmax(), samples_allphases.shape)
    cape_mode = samples_allphases.bins_cape[idx]
    subsat_mode = samples_allphases.bins_subsat[idy] 
    ax.plot(subsat_mode, cape_mode, 's', ms=4, color='grey')
    
    ## configuration of tick, labels
    ax.text(-1.2,4.3,'{}'.format(phase),fontsize=9,color='k')
    ax.grid(ls=':', lw=0.5, zorder=1)
    ax.set_xticks([-2,0,2,4,6,8,10,12])
    ax.set_yticks([-8,-6,-4,-2,0,2,4])
    ax.tick_params(labelsize=9)
    ax.tick_params('both', length=0, width=1, which='major')
    
    # set x-y domain 
    ax.set_ylim([-8,6])
    ax.set_xlim([-2,14])  
    
ax2.tick_params('both', length=0, width=1, which='major')
    
ax1.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax4.set_ylabel('B$_{L,cape}$ (K)',fontsize=9.5, labelpad=0)
ax4.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
ax5.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)
ax6.set_xlabel('B$_{L,subsat}$ (K)',fontsize=9.5, labelpad=0)

ax1.set_xticklabels([])
ax2.set_xticklabels([])
ax2.set_yticklabels([])
ax3.set_xticklabels([])
ax3.set_yticklabels([])
ax5.set_yticklabels([])
ax6.set_yticklabels([])

#====== schematic part for axis 2 =======
# add buoyancy reference 
ax2.plot(np.linspace(-2,16,13),np.linspace(-2,16,13), color='k', lw=1)
ax2.plot(np.linspace(-2,16,13),np.linspace(-2,16,13)-2, color='grey', ls='--',lw=0.5)
ax2.plot(np.linspace(-2,16,13),np.linspace(-2,16,13)-4, color='grey', ls='--',lw=0.5)
ax2.plot(np.linspace(-2,16,13),np.linspace(-2,16,13)-6, color='grey', ls='--',lw=0.5)

(idx,idy) = unravel_index(samples_allphases.argmax(), samples_allphases.shape)
cape_mode = samples_allphases.bins_cape[idx]
subsat_mode = samples_allphases.bins_subsat[idy] 
ax2.plot(subsat_mode, cape_mode, 's', ms=4, color='grey')

ax2.set_ylim([-8,6])
ax2.set_xlim([-2,14])

plt.suptitle('Fraction of occurrences, MCS grids, All, Duration: 12-24H',fontsize=10, fontweight='bold',y=0.97);
plt.tight_layout(h_pad=0.4,w_pad=0.01)

plt.show()

fig.savefig('/scratch/wmtsai/temp_mcs/fig/BL2Dhist_fraction_occurrence_allsurface.rotate.D12to24.pdf',
   bbox_inches='tight',transparent=False)

#### duration difference over oceans

In [ ]:
fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(6,4.5))

for p,(ax,phase,label_str) in enumerate(zip([ax1,ax2,ax3,ax4,ax5,ax6],samples_all.phase.values,
                                           ['(a)','(b)','(c)','(d)','(e)','(f)'])):
    
    gpm_sum = prec_gpm_sum.sel(duration_type=['SL','ML'], area_type='mcs', phase=phase).sum('duration_type')
    tmp = samples_all.sel(duration_type=['SL','ML'], area_type='mcs', phase=phase).sum('duration_type')
    tmp = tmp.where(tmp > 20)
    # conditional mean of gpm precip (mm/hr)
    cp = ax.pcolormesh(tmp.bins_subsat, tmp.bins_cape, gpm_sum/tmp, cmap='terrain_r', vmin=0, vmax=10, zorder=2)
    if phase == 'End':
        cbaxes = fig.add_axes([0.99, 0.3, 0.01, 0.4])              
        cbar = plt.colorbar(cp, cax=cbaxes, shrink=0.7)
        cbar.set_label('(mm/hr)')
    
    ax.set_title('{} {}'.format(label_str,phase),x=0.05,y=0.85,fontsize=9,color='k',loc='left')
    
    # contour highlighting light rainfall over low BL 
    ax.contour(tmp.bins_subsat, tmp.bins_cape, gpm_sum/tmp, levels=[2.5], colors=['limegreen'])
    
    # joint probability
    cf = ax.contour(tmp.bins_subsat, tmp.bins_cape, tmp/tmp.sum(), levels=np.linspace(0.0005,0.008,6), colors=['m'],
              linewidths=0.5, zorder=2)
    #ax.clabel(cf, cf.levels, fontsize=5, zorder=3)
    
    # add buoyancy reference 
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)+2, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)+4, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)+6, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='k', ls='-',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='k', ls='-',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)
    
    ax.set_yticks([-4,0,4,8,12])
    ax.set_xticks([-4,0,4,8,12])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    ax.set_ylim([-4,10])
    ax.set_xlim([-2,14])
    ax.grid(ls=':', lw=0.5, zorder=1)
    if p == 3 or p == 4 or p == 5:
        ax.set_xlabel('B$_{L,subsat}$ (K)',fontsize=10, labelpad=0)
        ax.set_xticklabels([-4,0,4,8,12])
    if p == 0 or p == 3:
        ax.set_ylabel('B$_{L,cape}$ (K)',fontsize=10, labelpad=0)
        ax.set_yticklabels([-4,0,4,8,12])
    ax.tick_params(labelsize=9)
    ax.tick_params('both', length=2, width=1, which='major')
    
    # add climatology joint PDF as reference
    #hist2d_clim_ocean = (samples_clim_ocean/samples_clim_ocean.sum())
    #hist2d_clim_ocean = hist2d_clim_ocean.where(hist2d_clim_ocean > 0)
    #cf = ax.contour(hist2d_clim_ocean.BL_subsat_bins, hist2d_clim_ocean.BL_cape_bins, hist2d_clim_ocean
    #                , levels=np.arange(0,0.015,0.0025), colors=['violet'],linewidths=1)
    #ax.clabel(cf, cf.levels, inline=True, fontsize=7)

plt.suptitle('Conditional GPM-precip, ocean, SL+ML, MCS grids \n (# samples in each bin > 100)',fontsize=10, fontweight='bold',y=0.97);
plt.tight_layout(h_pad=0.01,w_pad=0.01)

#fig.savefig('/scratch/wmtsai/temp_mcs/fig/BL2Dhist_condiGPMprecip_phase_ocean.png',dpi=300,
#         bbox_inches='tight',transparent=False)

In [ ]:
fig, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2,3,figsize=(6,4.5))

for p,(ax,phase,label_str) in enumerate(zip([ax1,ax2,ax3,ax4,ax5,ax6],samples_all.phase.values,
                                           ['(a)','(b)','(c)','(d)','(e)','(f)'])):
    
    gpm_sum = prec_gpm_sum.sel(duration_type=['LL','UL','UUL'], area_type='mcs', phase=phase).sum('duration_type')
    tmp = samples_all.sel(duration_type=['LL','UL','UUL'], area_type='mcs', phase=phase).sum('duration_type')
    tmp = tmp.where(tmp > 20)
    # conditional mean of gpm precip (mm/hr)
    cp = ax.pcolormesh(tmp.bins_subsat, tmp.bins_cape, gpm_sum/tmp, cmap='terrain_r', vmin=0, vmax=10, zorder=2)
    if phase == 'End':
        cbaxes = fig.add_axes([0.99, 0.3, 0.01, 0.4])              
        cbar = plt.colorbar(cp, cax=cbaxes, shrink=0.7)
        cbar.set_label('(mm/hr)')
    
    ax.set_title('{} {}'.format(label_str,phase),x=0.05,y=0.85,fontsize=9,color='k',loc='left')
    
    # contour highlighting light rainfall over low BL 
    ax.contour(tmp.bins_subsat, tmp.bins_cape, gpm_sum/tmp, levels=[2.5], colors=['limegreen'])
    
    # joint probability
    cf = ax.contour(tmp.bins_subsat, tmp.bins_cape, tmp/tmp.sum(), levels=np.arange(0,0.015,0.0025), colors=['m'],
              linewidths=1, zorder=2)
    #ax.clabel(cf, cf.levels, fontsize=5, zorder=3)
    
    # add buoyancy reference 
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13), color='k', lw=1.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)+2, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)+4, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)+6, color='grey', ls='--',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-2, color='k', ls='-',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-4, color='k', ls='-',lw=0.5)
    ax.plot(np.linspace(-2,14,13),np.linspace(-2,14,13)-6, color='grey', ls='--',lw=0.5)
    
    ax.set_yticks([-4,0,4,8,12])
    ax.set_xticks([-4,0,4,8,12])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    ax.set_ylim([-4,10])
    ax.set_xlim([-2,14])
    ax.grid(ls=':', lw=0.5, zorder=1)
    if p == 3 or p == 4 or p == 5:
        ax.set_xlabel('B$_{L,subsat}$ (K)',fontsize=10, labelpad=0)
        ax.set_xticklabels([-4,0,4,8,12])
    if p == 0 or p == 3:
        ax.set_ylabel('B$_{L,cape}$ (K)',fontsize=10, labelpad=0)
        ax.set_yticklabels([-4,0,4,8,12])
    ax.tick_params(labelsize=9)
    ax.tick_params('both', length=2, width=1, which='major')
    
    # add climatology joint PDF as reference
    #hist2d_clim_ocean = (samples_clim_ocean/samples_clim_ocean.sum())
    #hist2d_clim_ocean = hist2d_clim_ocean.where(hist2d_clim_ocean > 0)
    #cf = ax.contour(hist2d_clim_ocean.BL_subsat_bins, hist2d_clim_ocean.BL_cape_bins, hist2d_clim_ocean
    #                , levels=np.arange(0,0.015,0.0025), colors=['violet'],linewidths=1)
    #ax.clabel(cf, cf.levels, inline=True, fontsize=7)

plt.suptitle('Conditional GPM-precip, ocean, UL+UUL, MCS grids \n (# samples in each bin > 100)',fontsize=10, fontweight='bold',y=0.97);
plt.tight_layout(h_pad=0.01,w_pad=0.01)

#fig.savefig('/scratch/wmtsai/temp_mcs/fig/BL2Dhist_condiGPMprecip_phase_ocean.png',dpi=300,
#          bbox_inches='tight',transparent=False)

### whether MCSs over land actually produce weaker rainfalls
- simply 1-D precipitation histogram for mcs grids
- expecting higher probability of strong rain rates within MCSs over oceans compared to land
- 1-D Buoy-Precip over land and oceans? same critical pick-up but lower rain rate over land? coming from lower CWV?

In [ ]:
out_dir = Path('/scratch/wmtsai/temp_mcs/output_stats/')
precip_hist = xr.open_dataset(out_dir / 'MCS_precip_intensity_pdf_landsea.nc')
bins_precip = precip_hist.bins_precip
phist_oce = precip_hist.sel(surface_type='ocean').samples
phist_lnd = precip_hist.sel(surface_type='land').samples

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(4,4))

ax.plot(bins_precip, phist_oce/phist_oce.sum(), lw=1.5, marker='o',ms=5, color='b', label='Ocean')
ax.plot(bins_precip, phist_lnd/phist_lnd.sum(), lw=1.5, marker='o',ms=5, color='limegreen', label='Land')
ax.set_yscale('log')
ax.set_xlabel('Precipitation rate (mm/hr)', fontsize=10)
ax.set_ylabel('Log(Probabolity)', fontsize=10)
ax.set_title('Frequency of precipitation intensity \n Tropical MCS (30S-30N), 0.25-deg. GPM', fontsize=10)
ax.set_ylim([1e-5, 1])
ax.set_xlim([0,100])
ax.grid(lw=1, ls=':')
ax.legend()
plt.show()

fig.savefig('/scratch/wmtsai/temp_mcs/fig/PDF_precipitation_intensity_landsea.pdf',
           bbox_inches='tight',transparent=False)